### 참고한 사이트 및 코드

In [ ]:
# https://yamalab.tistory.com/64
req = requests.get('https://music.bugs.co.kr/chart')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml
list_song = soup.find_all(name="p", attrs={"class":"title"})
list_artist = soup.find_all(name="p", attrs={"class":"artist"})

# 곡명 추출
for index in range(0, len(list_song)):
    title = list_song[index].find('a').text
    print(index+1, ' : ', title)
    if index == 100:
        break

# 피처링 제거
for index in range(0, len(list_song)):
    title = list_song[index].find('a').text
    print(index+1, ' : ', title.split("(")[0])
    if index == 100:
        break

# csv로 저장
import csv

with open('melon_chart.csv', 'w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['rank', 'song', 'artist'])
    for index in range(0, len(list_song)):
        title = list_song[index].find('a').text
        artist = list_artist[index].find('a').text
        writer.writerow([index+1, title, artist])
        if index == 100:
            break

# 저장된 파일 pd로 읽기
import pandas as pd
datas = pd.read_csv('melon_chart.csv')

### bugs crawler (1 genre) : 돌려봤습니다~

In [19]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import display
import pandas as pd
import datetime as dt

In [20]:
## 0.start : 벅스 뮤직 PD로 접속
req = requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml

## 1.뮤직스타일 링크 추출 -> dict으로 저장
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[0].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
    
## 2. 뮤직스타일 링크로 삽입 
for url_categorytheme in dict_categorytheme.values():
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')

#### '신나는' 장르에서만

In [23]:
dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S') 

'2021-06-03 11:13:37'

In [25]:
### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : 신나는 장르에서만!!
req_category = requests.get('https://music.bugs.co.kr/musicpd?tag_id=429')
html_category = req_category.content
soup_category = BeautifulSoup(html_category, 'lxml')

### 2-2. page iteration
total_num = soup_category.select_one('p.desc').get_text()
total_num = int(re.sub(r'\D', '', total_num))
page_num = total_num//24 + 2
base_url = soup.select('div.paging > a')[0]['href'][:-1] 

album_count=0 

dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for n_th_page in range(1,page_num): 

    url_categorypages=base_url+f'{n_th_page}'
    dict_categoryalbum={}
    req_pages = requests.get(url_categorypages)
    html_pages = req_pages.content
    soup_pages = BeautifulSoup(html_pages, 'lxml')

    for album_tag in soup_pages.find_all('a', class_="title hyrend"):
        dict_categoryalbum[album_tag['title']]=album_tag['href']

    for url_categoryalbum in dict_categoryalbum.values():
        album_count+=1 
        
        req_album = requests.get(url_categoryalbum)
        html_album = req_album.content
        soup_album = BeautifulSoup(html_album, 'lxml')
        ele_album_style = str()

        for a_tag in soup_album.select('div.tag > a.hyrend'):
            ele_album_style+=a_tag.get_text()

        for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}):
            dict_category['category_theme'].append('신나는')
            dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
            dict_category['track_title'].append(music_tag['track_title'])
            dict_category['album_style'].append(ele_album_style)

    if n_th_page%5==0:
        print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
        display(pd.DataFrame.from_dict(dict_category).tail(1))

5번째 페이지 앨범120개


,category_theme,artist_disp_nm,track_title,album_style
2559,신나는,Riton(리톤),Friday (Dopamine Re-Edit) (feat. Mufasa & Hype...,#해외#팝송#해외 팝#여름#초여름#신나는#시원한#드라이브


10번째 페이지 앨범240개


,category_theme,artist_disp_nm,track_title,album_style
6341,신나는,Stephanie Poetri,Selfish,#국내외#멍때릴때#지루할때#공강


15번째 페이지 앨범360개


...

,category_theme,artist_disp_nm,track_title,album_style
53711,신나는,Shanuka,Pain,#해외#팝송#해외 팝#밤/새벽#그루브한


105번째 페이지 앨범2520개


,category_theme,artist_disp_nm,track_title,album_style
57252,신나는,Melody Gardot(멜로디 가르도트),Moon River,#재즈#국내외#분위기있는#와인#호텔/바#저녁


110번째 페이지 앨범2640개


,category_theme,artist_disp_nm,track_title,album_style
59948,신나는,Fun.(펀.),We Are Young (feat. Janelle Monae),#해외#락#이색테마#명곡콜렉션


In [26]:
dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2021-06-03 12:10:15'

장르 내 페이지 수나 앨범 갯수에 따라 다를 것같은데 '신나는' 1개 장르 돌릴 때 60분 정도 걸려요.. 

In [27]:
pd.DataFrame.from_dict(dict_category).shape

(59949, 4)

In [28]:
display(pd.DataFrame.from_dict(dict_category))

,category_theme,artist_disp_nm,track_title,album_style
0,신나는,Hauskey,Somewhere,#해외#팝송#해외 팝#아침#선선한#등교/출근길#기분전환#출근/퇴근길#일상#산책/여행...
1,신나는,Dempsey Hope,girl of my dreams,#해외#팝송#해외 팝#아침#선선한#등교/출근길#기분전환#출근/퇴근길#일상#산책/여행...
2,신나는,bülow,First Place,#해외#팝송#해외 팝#아침#선선한#등교/출근길#기분전환#출근/퇴근길#일상#산책/여행...
3,신나는,The Millennial Club,feel the same,#해외#팝송#해외 팝#아침#선선한#등교/출근길#기분전환#출근/퇴근길#일상#산책/여행...
4,신나는,Devin Kennedy,Something Bout That Feeling,#해외#팝송#해외 팝#아침#선선한#등교/출근길#기분전환#출근/퇴근길#일상#산책/여행...
...,...,...,...,...
59944,신나는,Green Day(그린 데이),American Idiot,#해외#락#이색테마#명곡콜렉션
59945,신나는,Linkin Park(린킨 파크),In The End,#해외#락#이색테마#명곡콜렉션
59946,신나는,Muse(뮤즈),Time Is Running Out,#해외#락#이색테마#명곡콜렉션
59947,신나는,My Chemical Romance(마이 케미컬 로맨스),Welcome To The Black Parade,#해외#락#이색테마#명곡콜렉션


In [30]:
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_genre_신나는.csv', index = False, encoding = 'UTF-8')

### bugs crawler (total genre) : 아직 무서워서 못 돌려봤어요!

In [ ]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import display
import pandas as pd
import datetime as dt
#dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S') 

In [ ]:
## 0.start : 벅스 뮤직 PD로 접속
req = requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml

## 1.뮤직스타일 링크 추출 -> dict으로 저장
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[0].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
dict_categorytheme

In [ ]:
## 2. 뮤직스타일 링크로 삽입 
for name_categorytheme,url_categorytheme in dict_categorytheme.items():
    print(name_categorytheme,url_categorytheme) # : 22개 장르 모두 나오는 것 확인
    ### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : name_categorytheme 장르에서만!!
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml')
    #print(soup_category)

    ### 2-2. page iteration
    total_num = soup_category.select_one('p.desc').get_text()
    total_num = int(re.sub(r'\D', '', total_num))
    page_num = total_num//24 + 2

    ### 몇 페이지인지
    base_url = soup.select('div.paging > a')[0]['href'][:-1] # https://music.bugs.co.kr/musicpd?order=list&page=
    album_count=0 #앨범갯수 맞추기용 상수
    dict_category = {'category_theme':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        req_pages = requests.get(url_categorypages)
        html_pages = req_pages.content
        soup_pages = BeautifulSoup(html_pages, 'lxml')

        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        # print('dict_categoryalbum',dict_categoryalbum) # 순서대로는 아닌데 쨋든 album dict 완료 
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1 #앨범갯수 맞추기용 상수
            req_album = requests.get(url_categoryalbum)
            html_album = req_album.content
            soup_album = BeautifulSoup(html_album, 'lxml') # pip install lxml
            ## 앨범 특징 : 팀원 피드백) 곡 제목-스타일-음악에 대표되는 감성? 을 위주로 중 음악에 대표되는 감성을 앨범 #로 태그해놔서 가져와봤어요
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_theme'].append(name_categorytheme)
            # 수작업으로 이렇게 했는데 추후에 dict에서 key 빼서 할 수도 있을 것같아요. 근데 22개 장르라 그냥 수작업으로 해도 괜찮을 것같고 그렇습니다아 category
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)

        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(1))

In [ ]:
pd.DataFrame.from_dict(dict_category).shape

In [ ]:
display(pd.DataFrame.from_dict(dict_category))

In [ ]:
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_total_genre.csv', index = False, encoding = 'UTF-8')